In [25]:
import time
import azure.cognitiveservices.speech as speechsdk

text = []

def speech_recognize_continuous_from_file():
    speech_config = speechsdk.SpeechConfig(subscription="268d8c1496a44cc0b7c044ec33edbcd5", region="southcentralus")
    audio_config = speechsdk.audio.AudioConfig(filename="../data/sampledata_audiofiles_speechService.wav")

    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, language="en-US", audio_config=audio_config)

    done = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def recognized(evt: speechsdk.SpeechRecognitionEventArgs):
        text.append(evt.result.text)
        print('RECOGNIZED: {}'.format(evt.result.text))
    
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

In [26]:
speech_recognize_continuous_from_file()

SESSION STARTED: SessionEventArgs(session_id=deb5e4dae86f4c29a300b6e177a9b05e)
RECOGNIZED: What is the Speech Services? The Speech Service is the unification of speech to text, text to speech and speech translation into a single Azure subscription. It is easy to speech enable your applications, tools and devices with the Speech SDK, Speech Devices SDK, and the Restful APIs.
RECOGNIZED: These features made-up the Speech services. Use the links in this table to learn more about common use cases for each feature, or browse the API references.
RECOGNIZED: Speak to text, transcribe or translate audio streams or local files to text in real time that your applications, tools, or devices can consume or display.
RECOGNIZED: Use speech to text with language understanding to derive user intents from the transcribe speech and act on voice commands.
CANCELED SpeechRecognitionCanceledEventArgs(session_id=deb5e4dae86f4c29a300b6e177a9b05e, result=SpeechRecognitionResult(result_id=eb481b999111402998c7e

In [27]:
print(text)

['What is the Speech Services? The Speech Service is the unification of speech to text, text to speech and speech translation into a single Azure subscription. It is easy to speech enable your applications, tools and devices with the Speech SDK, Speech Devices SDK, and the Restful APIs.', 'These features made-up the Speech services. Use the links in this table to learn more about common use cases for each feature, or browse the API references.', 'Speak to text, transcribe or translate audio streams or local files to text in real time that your applications, tools, or devices can consume or display.', 'Use speech to text with language understanding to derive user intents from the transcribe speech and act on voice commands.']


In [28]:
with open ('../data/text.txt', 'w') as f:
    f.writelines(text)